<a href="https://colab.research.google.com/github/QuasimRizvi/BigData/blob/main/BigdataNotebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [4]:
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [6]:
import findspark
findspark.init()

In [7]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-31 15:33:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.198.117.197, 174.129.26.222, 35.175.47.233, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.198.117.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  44.0MB/s    in 0.3s    

2021-03-31 15:33:35 (44.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  

In [9]:
ls

ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7/
sample_data/                  spark-3.1.1-bin-hadoop2.7.tgz


In [10]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col

In [11]:
project_id = 'api3-307502'

In [12]:
from google.colab import auth
auth.authenticate_user()

In [13]:
!gcloud config set project {project_id}

Updated property [core/project].


In [14]:
!gsutil cp gs://ethereum-bigdata/token_transfer* /tmp/

Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000000...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000001...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000002...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000003...
- [4 files][  2.0 GiB/  2.0 GiB]   58.3 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000004...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000005...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000006...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_00000000000

In [15]:
spark = SparkSession(sc)

In [16]:
token_transfer = os.listdir('/tmp')

In [17]:
token_transfer = ["/tmp/"+x for x in token_transfer if "token_transfer_" in x]
token_transfer.sort()
token_transfer

['/tmp/token_transfer_2019_11_01__2020_01_01_000000000000',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000001',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000002',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000003',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000004',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000005',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000006',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000007',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000008',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000009',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000010',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000011',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000012',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000013',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000014',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000015',
 '/tmp/token_transfer_2019_11_01__2020_0

In [18]:
from functools import reduce
dfToken_transfer = reduce(lambda x,y: x.unionAll(y), 
            [spark.read.format('com.databricks.spark.csv')
                       .load(f, header="true", inferSchema="true") 
             for f in token_transfer])

In [19]:
dfToken_transfer.createOrReplaceTempView("tokentransfer")
spark.sql("SELECT COUNT(*) FROM tokentransfer").show()

+---------+
| count(1)|
+---------+
|167801660|
+---------+



In [22]:
dfToken_transfer.printSchema()

root
 |-- token_address: string (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- value: double (nullable = true)
 |-- transaction_hash: string (nullable = true)
 |-- log_index: integer (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: integer (nullable = true)



In [23]:
dfToken_transfer.take(5)

[Row(token_address='0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2', from_address='0x29098405fe3796251b9198a5c6475d7eb8c38dcd', to_address='0xc54df9fbe4212289ccb4d08546ba928cec7f9426', value=6983566899456733.0, transaction_hash='0xbbb6b63c201cfa43a96839f06a3e0352a05269a7fd37c0f7c6c46d4aeb1df35f', log_index=372, block_timestamp='2019-11-28 06:39:12 UTC', block_number=9014085),
 Row(token_address='0x457476bc97adef10aba63fcadaefe503553fa0d2', from_address='0x7ece946a162c2942072b3f31cc5e67c7c777313f', to_address='0x5acaf5ea106ba170a83466d238bc60cea2caaf6c', value=1000000000000.0, transaction_hash='0x0cfac85375af9c6305c0fcc040c2d2ff07a07a223ca5840e06515ab047320f43', log_index=69, block_timestamp='2019-11-28 13:08:00 UTC', block_number=9015625),
 Row(token_address='0x0cf0ee63788a0849fe5297f3407f701e122cc023', from_address='0x95de091a576a0a6caed5a888751acf9620ca18d0', to_address='0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be', value=1.0000843e+22, transaction_hash='0x3aac0b9d97ceaba8e32ac89708192c05

In [25]:
!gsutil cp gs://ethereum-bigdata/tokens_* /tmp/

Copying gs://ethereum-bigdata/tokens_2019_11_01...
/ [1 files][  6.5 MiB/  6.5 MiB]                                                
Operation completed over 1 objects/6.5 MiB.                                      


In [26]:
tokens = os.listdir('/tmp')

In [27]:
tokens = ["/tmp/"+x for x in tokens if "tokens" in x]
tokens.sort()
tokens

['/tmp/tokens_2019_11_01']

In [28]:
from functools import reduce
dfTokens = reduce(lambda x,y: x.unionAll(y), 
            [spark.read.format('com.databricks.spark.csv')
                       .load(f, header="true", inferSchema="true") 
             for f in tokens])

In [30]:
dfTokens.createOrReplaceTempView("tokens")
spark.sql("SELECT COUNT(*) FROM tokens").show()

+--------+
|count(1)|
+--------+
|   37992|
+--------+



In [31]:
dfTokens.printSchema()

root
 |-- address: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- name: string (nullable = true)
 |-- decimals: string (nullable = true)
 |-- total_supply: string (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: string (nullable = true)
 |-- block_hash: string (nullable = true)



In [32]:
dfTokens.take(5)

[Row(address='0xad03473a02210fd53e3c44fbbda3e27f2314691a', symbol=None, name=None, decimals=None, total_supply=None, block_timestamp='2019-11-23 17:48:16 UTC', block_number='8987748', block_hash='0xd8a7e145f7cbd90ebad496d97344bde66fc2e1ae70849eb331ffc6a2e1732370'),
 Row(address='0x2acdbcbfea81d242f364f8d97ab2bf04c7d3946d', symbol=None, name=None, decimals=None, total_supply=None, block_timestamp='2019-11-18 16:15:00 UTC', block_number='8957431', block_hash='0x8b2ad641e100acbed18cff18e32283324c13a6c0b982fe9007bdd29af9d8d220'),
 Row(address='0x040b27772cd7e7337186619f6d66d4308ba6b068', symbol=None, name=None, decimals=None, total_supply=None, block_timestamp='2019-11-08 03:00:13 UTC', block_number='8893758', block_hash='0xa87fb5f780e1e16986abae6734bbb9dc7e7ace119f315fa5faff22419918405b'),
 Row(address='0xe35257f56ee409523616b15179635b7e05087a80', symbol=None, name=None, decimals=None, total_supply=None, block_timestamp='2019-11-17 15:21:35 UTC', block_number='8951159', block_hash='0xf49d

In [33]:
!gsutil cp gs://ethereum-bigdata/blocks_* /tmp/

Copying gs://ethereum-bigdata/blocks_2019_11_01_000000000000...
Copying gs://ethereum-bigdata/blocks_2019_11_01_000000000001...
Copying gs://ethereum-bigdata/blocks_2019_11_01_000000000002...
Copying gs://ethereum-bigdata/blocks_2019_11_01_000000000003...
\ [4 files][  2.0 GiB/  2.0 GiB]   67.4 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://ethereum-bigdata/blocks_2019_11_01_000000000004...
Copying gs://ethereum-bigdata/blocks_2019_11_01_000000000005...
Copying gs://ethereum-bigdata/blocks_2019_11_01_000000000006...
- [7 files][  3.4 GiB/  3.4 GiB]   50.9 MiB/s                                   
Operation completed over 7 objects/3.4 GiB.                                      


In [34]:
blocks = os.listdir('/tmp')

In [35]:
blocks = ["/tmp/"+x for x in blocks if "blocks" in x]
blocks.sort()
blocks

['/tmp/blocks_2019_11_01_000000000000',
 '/tmp/blocks_2019_11_01_000000000001',
 '/tmp/blocks_2019_11_01_000000000002',
 '/tmp/blocks_2019_11_01_000000000003',
 '/tmp/blocks_2019_11_01_000000000004',
 '/tmp/blocks_2019_11_01_000000000005',
 '/tmp/blocks_2019_11_01_000000000006']

In [36]:
from functools import reduce
dfBlocks = reduce(lambda x,y: x.unionAll(y), 
            [spark.read.format('com.databricks.spark.csv')
                       .load(f, header="true", inferSchema="true") 
             for f in blocks])

In [37]:
dfBlocks.createOrReplaceTempView("blocks")
spark.sql("SELECT COUNT(*) FROM blocks").show()

+--------+
|count(1)|
+--------+
| 3292493|
+--------+



In [38]:
dfBlocks.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- hash: string (nullable = true)
 |-- parent_hash: string (nullable = true)
 |-- nonce: string (nullable = true)
 |-- sha3_uncles: string (nullable = true)
 |-- logs_bloom: string (nullable = true)
 |-- transactions_root: string (nullable = true)
 |-- state_root: string (nullable = true)
 |-- receipts_root: string (nullable = true)
 |-- miner: string (nullable = true)
 |-- difficulty: long (nullable = true)
 |-- total_difficulty: decimal(23,0) (nullable = true)
 |-- size: integer (nullable = true)
 |-- extra_data: string (nullable = true)
 |-- gas_limit: integer (nullable = true)
 |-- gas_used: integer (nullable = true)
 |-- transaction_count: integer (nullable = true)



In [39]:
dfBlocks.take(5)

[Row(timestamp='2021-01-08 05:29:58 UTC', number=11612121, hash='0x0073e6f310bf666e16e8d429e8ecdc26850f2f7ed858a6cc87cb19f3a75dbc89', parent_hash='0xd42498354f7466cc917318f222866848af142212b7f6b036ab9bbbe6f4c6071f', nonce='0xf3f9a9668b2464a9', sha3_uncles='0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347', logs_bloom='0xa82a11aec8a008471c49b880e0895a006208800d86d04018100504e564008015e900012060110a0100411001001919388290985058e418510378c010b238c348980204615218f65c680281a8a0c0c2a0108500038c40680128000c81f36231007018082c0220b44c452848800442888963005102502946105822341110041c0080088987d2003e046a44106908209c4e3302885511007098413320410b18080a0fb4dc201a1420320400809c1140e89100080014242ae0a40406311702062078154a2ea3c84c10044a04900913e8124640b84300108da814800170928410642a8cb2a1505084112875010085883980b002a40040280006494291430d0b09340a', transactions_root='0x4268a77e3be6220098d538d20f08dc8dd3b98ff4a77b2f44866539b4edfde3cf', state_root='0x59cf22533a2cc22cea20f7ae55265297f1b6bfb8f57